for Channel codes

In [ ]:
import os
import time
from flask import Flask, render_template, request
from PIL import Image
import cv2
import numpy as np
import tensorflow as tf
from werkzeug.utils import secure_filename  # Add this import

app = Flask(__name__)

# Load the saved model
model = tf.keras.models.load_model('C:/Users/user1/Desktop/APP TEST/model_DenseNet169_ECA_Net_2.h5')

# Define the classes
classes = ['normal','polyps']

# Define the upload folder
upload_folder = 'C:/Users/user1/Desktop/APP TEST/static'

# Ensure the upload folder exists
if not os.path.exists(upload_folder):
    os.makedirs(upload_folder)

# Define the function to preprocess the image
def preprocess_image(input_path, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Define the target size for resampling
    target_size = (224, 224)

    # Open the image
    img = Image.open(input_path)

    # Resample the image using Lanczos interpolation
    img_resized = img.resize(target_size, resample=Image.LANCZOS)

    # Split the image channels
    b, g, r = cv2.split(np.array(img_resized))

    # Apply CLAHE to each channel separately
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_b = clahe.apply(b)
    enhanced_g = clahe.apply(g)
    enhanced_r = clahe.apply(r)

    # Merge the enhanced channels back into an RGB image
    enhanced = cv2.merge([enhanced_b, enhanced_g, enhanced_r])

    # Apply sharpening using the unsharp mask technique
    blur = cv2.GaussianBlur(enhanced, (0, 0), 3)
    sharp = cv2.addWeighted(enhanced, 1.5, blur, -0.5, 0)

    # Convert the sharpened image back to PIL Image
    sharp_pil = Image.fromarray(sharp)

    # Generate the output file path
    output_filename = os.path.basename(input_path)
    output_path = os.path.join(output_folder, output_filename)

    # Save the pre-processed image to the output folder
    sharp_pil.save(output_path)

    # Close the image
    img.close()

    return np.array(sharp_pil)


# Function to compute Grad-CAM for channel attention layer
def compute_grad_cam_channel_attention(model, image, layer_name):
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(layer_name).output, model.output])

    # Pass the input image through the channel attention layer
    with tf.GradientTape() as tape:
        channel_attention_output, predictions = grad_model(image)
        loss = predictions[:, np.argmax(predictions[0])]

    grads = tape.gradient(loss, channel_attention_output)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    channel_attention_output = channel_attention_output[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, channel_attention_output), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    return heatmap


def custom_color_map(heatmap):
    # Scale the heatmap to fit within the 8-bit range (0-255)
    heatmap = (heatmap * 255).astype(np.uint8)

    # Apply the 'JET' color map to the scaled heatmap
    colormap = cv2.COLORMAP_JET
    heatmap_colored = cv2.applyColorMap(heatmap, colormap)

    return heatmap_colored



# Function to overlay the activation map on the image
def overlay_activation_map(image, activation_map):
    # Normalize the heatmap to [0, 1]
    heatmap = activation_map - np.min(activation_map)
    heatmap /= np.max(heatmap)

    # Apply the custom color map to the heatmap
    heatmap_colored = custom_color_map(heatmap)

    # Resize the heatmap_colored to match the size of the original image
    heatmap_resized = cv2.resize(heatmap_colored, (image.shape[1], image.shape[0]))

    # Overlay the heatmap on the image
    overlay = cv2.addWeighted(image, 0.5, heatmap_resized, 0.7, 0.1)
    return overlay


ALLOWED_EXTENSIONS = {'png', 'jpeg', 'jpg'}

# Function to check if the uploaded file has an allowed extension
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/classify', methods=['POST'])
def classify():
    # Check if the request contains a file
    if 'image' not in request.files:
        return render_template('index.html', error='No image file found, please upload an image')

    # Get the uploaded file
    uploaded_file = request.files['image']

    # Check if the file has a filename
    if uploaded_file.filename == '':
        return render_template('index.html', error='No image selected, please upload an image')

    # Check if the file has an allowed extension
    if not allowed_file(uploaded_file.filename):
        return render_template('index.html', error='Invalid file format. Only .png, .jpeg and .jpg formats are allowed.')

    # Save the uploaded file to the upload folder
    filename = secure_filename(uploaded_file.filename)
    image_path = os.path.join(upload_folder, filename)
    uploaded_file.save(image_path)

    # Preprocess the input image
    preprocess_image(image_path, upload_folder)

    # Load the pre-processed image
    preprocessed_image = Image.open(os.path.join(upload_folder, os.path.basename(image_path)))

    # Convert the image to a numpy array
    preprocessed_image_array = np.array(preprocessed_image)

    # Check if the preprocessed image is valid
    if preprocessed_image_array is None:
        return render_template('index.html', error='Error processing the uploaded image. Please try again.')

    try:
        # Start the timer
        start_time = time.time()
        # Perform the prediction
        input_tensor = np.expand_dims(preprocessed_image_array / 255.0, axis=0)
        predictions = model.predict(input_tensor)
        predicted_class_index = np.argmax(predictions)
        predicted_class = classes[predicted_class_index]
        prediction_accuracy = predictions[0][predicted_class_index]

        # Get the Grad-CAM activation map for channel attention layer
        activation_map = compute_grad_cam_channel_attention(model, input_tensor, layer_name='tf.math.multiply_1')  

        # Overlay the activation map on the image
        overlay_image = overlay_activation_map(preprocessed_image_array, activation_map)

        # Save the overlay image
        overlay_path = os.path.join(upload_folder, 'overlay.png')
        cv2.imwrite(overlay_path, overlay_image)

        # Stop the timer and calculate the elapsed time
        end_time = time.time()
        elapsed_time = end_time - start_time

        # Update the template with the prediction results
        return render_template('index.html', image_path=os.path.basename(image_path),
                               grad_cam_path='overlay.png', predicted_class=predicted_class,
                               prediction_accuracy=prediction_accuracy,elapsed_time=elapsed_time)

    except Exception as e:
        # Handle any potential exceptions during visualization
        return render_template('index.html', error='Error during visualization: {}'.format(str(e)))

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Nov/2023 23:30:14] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 8s 8s/step


127.0.0.1 - - [26/Nov/2023 23:30:29] "POST /classify HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2023 23:30:29] "GET /static/test_polyps__14.jpg HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2023 23:30:29] "GET /static/overlay.png HTTP/1.1" 200 -


1/1 [==============================] - 0s 41ms/step


...

127.0.0.1 - - [26/Nov/2023 23:39:51] "POST /classify HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2023 23:39:51] "GET /static/test_normal__24.jpg HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2023 23:39:51] "GET /static/overlay.png HTTP/1.1" 200 -


1/1 [==============================] - 0s 36ms/step


127.0.0.1 - - [26/Nov/2023 23:41:44] "POST /classify HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2023 23:41:44] "GET /static/images_7.jpg HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2023 23:41:44] "GET /static/overlay.png HTTP/1.1" 200 -


1/1 [==============================] - 0s 39ms/step


127.0.0.1 - - [26/Nov/2023 23:42:40] "POST /classify HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2023 23:42:40] "GET /static/download.jpg HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2023 23:42:40] "GET /static/overlay.png HTTP/1.1" 200 -
